In [1]:
import matplotlib.pyplot as plt

import re, os, time, random
import pandas as pd
import numpy as np

import tensorflow as tf
import pickle as pkl

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.models import Model, load_model
from keras.layers import LSTM, GRU, Input, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences

from prettytable import PrettyTable
from nltk.translate.bleu_score import sentence_bleu

In [2]:
SEED = 1337

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Подготовка даных

In [3]:
def preprocess_sentence(data, punctuation=False, add_tokens=False):
    """
        Функция для предобработки 
    """
    
    # Уменьшаем регистр и убираем лишние пробелы
    data = [w.lower().strip() for w in data]
    
    # Замена всех символов 'æ' на однотипный
    data = [re.sub(r"ӕ", r"æ", w) for w in data]
    
    # Удаление апострофом
    data = [re.sub("'", '', w) for w in data]
    
    if punctuation:
        # Делаем между словом и знаком пунктуации отступ 'слово! -> слово !'
        data = [re.sub(r"([?.!,])", r" \1 ", w) for w in data]
        data = [re.sub(r'[" "]+', " ", w) for w in data]
    else:
        # Удаляет все знаки пунктуации
        data = [re.sub(r"[^\w\s]", r"", w) for w in data]
    
    # Выкидываем все остальные символы из рассмотрения 
    data = [re.sub(r"[^a-яА-Яa-zA-Z?.!,æё]+", " ", w) for w in data]
    data = [w.rstrip().strip() for w in data]
    
    # Добавляем токены для начала и конца предложения
    if add_tokens:
        data = [f'{start_target} {w} {end_target}' for w in data]
        
    return data

In [4]:
def read_dataset(path, preparing=False):
    """
        Функция для создания датасета
    """
    
    # Открытие файла с данными в кодировке UTF-8
    with open(path, encoding='utf-8') as f:
        data = f.read()
    
    # Разделение данных на пары <<предложение, перевод>>
    uncleaned_data_list = data.split('\n')
    
    source_word = []
    target_word = []
    for word in uncleaned_data_list:
        source_word.append(word.split('\t')[0])
        target_word.append(word.split('\t')[1])
    
    # Инецализация датафрейма pandas
    language_data = pd.DataFrame(columns=['Source','Target'])
    if preparing:
        language_data['Source'] = preprocess_sentence(source_word)
        language_data['Target'] = preprocess_sentence(target_word, add_tokens=True)
    else:
        language_data['Source'] = source_word
        language_data['Target'] = target_word
        
    return language_data

In [5]:
def tokenize(text_data):
    '''
        Токенайзер
    '''
    
    tokenizer = Tokenizer(filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(text_data)
    return tokenizer

In [6]:
def preparing_data(path):
    data = read_dataset(path, preparing=True)
    tokenizer_input, tokenizer_output = tokenize(data['Source'].values), tokenize(data['Target'].values)
    input_max_length, output_max_length = len(tokenizer_input.word_index) + 1, len(tokenizer_output.word_index) + 1
    
    return data, tokenizer_input, tokenizer_output, input_max_length, output_max_length

# Построение модели Encoder-Decoder

In [7]:
class Encoder(tf.keras.Model):
    """
        Энкодер 
    """
    
    def __init__(self, vocab_size_input, HIDDEN_DIM):
        super(Encoder, self).__init__()
        
        self.inputs = Input(shape=(None,), name="encoder_inputs")
        self.embedding = Embedding(vocab_size_input, HIDDEN_DIM, mask_zero=True, name="encoder_embedding")(self.inputs)
        
        self.encoder = GRU(HIDDEN_DIM, return_state=True, name="encoder_gru")
        self.outputs, state_h = self.encoder(self.embedding)
        self.states = [state_h]
    
    @staticmethod
    def getModel(model):
        inputs_inf = model.input[0]
        outputs_inf, inf_state_h = model.layers[4].output
        inf_states = [inf_state_h]
        
        return Model(inputs_inf, inf_states, name='Encoder') 

In [8]:
class Decoder(tf.keras.Model):
    """
        Декодер 
    """
    
    def __init__(self, vocab_size_output, HIDDEN_DIM, encoder_states):
        super(Decoder, self).__init__()

        self.inputs = Input(shape=(None,), name="decoder_inputs")
        self.embedding = Embedding(vocab_size_output, HIDDEN_DIM, mask_zero=True, name="decoder_embedding")(self.inputs)
        
        self.decoder = GRU(HIDDEN_DIM, return_sequences=True, return_state=True, name="decoder_gru")
        self.outputs, _ = self.decoder(self.embedding, initial_state=encoder_states)
        self.dense = Dense(vocab_size_output, activation='softmax', name="dense_gru")
        self.outputs = self.dense(self.outputs)
    
    @staticmethod
    def getModel(model):
        state_h_input = Input(shape=(HIDDEN_DIM,))
        state_input = [state_h_input]

        input_inf = model.input[1]
        emb_inf = model.layers[3](input_inf)
        lstm_inf = model.layers[5]
        output_inf, state_h_inf = lstm_inf(emb_inf, initial_state=state_input)
        state_inf = [state_h_inf]
        dense_inf = model.layers[6]
        output_final = dense_inf(output_inf)

        return Model([input_inf]+state_input, [output_final]+state_inf, name='Decoder')

In [9]:
def generatorBatch(X, Y, batch_size):
    max_length = lambda data: max([len(x.split(' ')) for x in data])
    
    while True:
        for j in range(0, len(X), batch_size):
            encoder_data_input   = np.zeros((batch_size, max_length(X)), dtype='float32')
            decoder_data_input   = np.zeros((batch_size, max_length(Y)), dtype='float32')
            decoder_target_input = np.zeros((batch_size, max_length(Y), vocab_size_target), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j : j + batch_size], Y[ j : j + batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_data_input[i, t] = tokenizer_input.word_index[word]
                for t, word in enumerate(target_text.split()):
                    decoder_data_input[i, t] = tokenizer_output.word_index[word]
                    if t>0:
                        decoder_target_input[i,t - 1,tokenizer_output.word_index[word]] = 1
            yield ([encoder_data_input, decoder_data_input], decoder_target_input)

# Входные параметры

In [10]:
path = 'Date/rus-oss.txt'

start_target = "<sos>"
end_target = "<eos>"

HIDDEN_DIM = 256
batch_size = 6
epochs = 20

In [11]:
data, tokenizer_input, tokenizer_output, vocab_size_source, vocab_size_target = preparing_data(path)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data['Source'].values, data['Target'].values, test_size = 0.1)

In [13]:
read_dataset(path).head()

,Source,Target
0,Чего ты смеёшься?,Цæуыл худыс?
1,Этот нож очень острый.,Ацы кард тынг цыргъ у.
2,У кошки девять жизней.,Гæдыйæн фараст царды ис.
3,Сегодня облачно.,Абон у асæст.
4,Он был вождём своего племени 35 лет.,Уый йæ знæмы раздзог уыдис 35 азы дæргъы.


In [14]:
data.head()

,Source,Target
0,чего ты смеёшься,<sos> цæуыл худыс <eos>
1,этот нож очень острый,<sos> ацы кард тынг цыргъ у <eos>
2,у кошки девять жизней,<sos> гæдыйæн фараст царды ис <eos>
3,сегодня облачно,<sos> абон у асæст <eos>
4,он был вождём своего племени лет,<sos> уый йæ знæмы раздзог уыдис азы дæргъы <eos>


In [15]:
print(f'Тренировочная выборка: {len(X_train)}')
print(f'Тестовая выборка: {len(X_test)}')

Тренировочная выборка: 423
Тестовая выборка: 47


In [16]:
encoder = Encoder(vocab_size_source, HIDDEN_DIM)
decoder = Decoder(vocab_size_target, HIDDEN_DIM, encoder.states)

In [17]:
model = Model([encoder.inputs, decoder.inputs], decoder.outputs, name="Seq2Seq-GRU-Translation")

In [18]:
loss_function = 'categorical_crossentropy'
model.compile(loss=loss_function, optimizer='rmsprop', metrics=['accuracy'])

In [32]:
model.layers[2]

# Обучение модели

In [19]:
# %%capture
model.fit(generatorBatch(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)//batch_size, epochs=epochs)

Epoch 1/20
70/70 [==============================] - 8s 30ms/step - loss: 1.2740 - accuracy: 0.1566
Epoch 2/20
70/70 [==============================] - 2s 30ms/step - loss: 1.1534 - accuracy: 0.1694
Epoch 3/20
70/70 [==============================] - 2s 31ms/step - loss: 1.1026 - accuracy: 0.1703
Epoch 4/20
70/70 [==============================] - 2s 30ms/step - loss: 1.0395 - accuracy: 0.1739
Epoch 5/20
70/70 [==============================] - 2s 30ms/step - loss: 0.9968 - accuracy: 0.1789
Epoch 6/20
70/70 [==============================] - 2s 30ms/step - loss: 0.9447 - accuracy: 0.1862
Epoch 7/20
70/70 [==============================] - 2s 31ms/step - loss: 0.8987 - accuracy: 0.1947
Epoch 8/20
70/70 [==============================] - 2s 30ms/step - loss: 0.8620 - accuracy: 0.2047
Epoch 9/20
70/70 [==============================] - 2s 30ms/step - loss: 0.8236 - accuracy: 0.2146
Epoch 10/20
70/70 [==============================] - 2s 30ms/step - loss: 0.7885 - accuracy: 0.2290
Epoch 11/

# Предсказание

In [20]:
def predict(input_seq, encoder_model, decoder_model, tokenizer_output, reverse_word_map_target):
    state_values_encoder = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tokenizer_output.word_index[start_target]
    stop_condition = False
    decoder_sentance = ''
    
    while not stop_condition:
        sample_word , decoder_h, decoder_c= decoder_model.predict([target_seq] + state_values_encoder)
        sample_word_index = np.argmax(sample_word[0,-1,:])
        decoder_word = reverse_word_map_target[sample_word_index]
        decoder_sentance += ' ' + decoder_word
        if (decoder_word == end_target or 
            len(decoder_sentance) > 70):
            stop_condition = True
        target_seq[0, 0] = sample_word_index
        state_values_encoder = [decoder_h,decoder_c]
    return decoder_sentance

# Тесты

In [21]:
encoder_model = Encoder.getModel(model)
decoder_model = Decoder.getModel(model)

In [22]:
reverse_word_map_input  = dict(map(reversed, tokenizer_input.word_index.items()))
reverse_word_map_target = dict(map(reversed, tokenizer_output.word_index.items()))

In [23]:
for i in range(10):
    sentance = X_train[i]
    original_target = y_train[i]
    input_seq = tokenizer_input.texts_to_sequences([sentance])
    pad_sequence = pad_sequences(input_seq, maxlen=30, padding='post')
    predicted_target = predict(pad_sequence, encoder_model, decoder_model, tokenizer_output, reverse_word_map_target)
    reference = original_target[5:-5].strip().split()
    hypothetic = predicted_target[:-5].strip().split()
    
    print(f'Test #{i + 1}')
    table = pd.Series([sentance, 
                       original_target[5:-5].strip(), 
                       predicted_target[:-5].strip(), 
                       "%.3f" % sentence_bleu([hypothetic], reference, weights = [1])], 
                          index=['Sentance','Original','Predicted', 'BLEU'])
    
    print(table.to_string())
    print("==" * 50)

ValueError: in user code:

    File "D:\Programms\Anaconda\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "D:\Programms\Anaconda\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\Programms\Anaconda\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "D:\Programms\Anaconda\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "D:\Programms\Anaconda\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\Programms\Anaconda\lib\site-packages\keras\engine\input_spec.py", line 200, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_name}" expects {len(inp

In [24]:
for i in range(10):
    sentance = X_test[i]
    original_target = y_test[i]
    input_seq = tokenizer_input.texts_to_sequences([sentance])
    pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
    predicted_target = predict(pad_sequence, encoder_model, decoder_model, tokenizer_output, reverse_word_map_target)
    reference = original_target[5:-5].strip().split()
    hypothetic = predicted_target[:-5].strip().split()
    
    print(f'Test #{i + 1}')
    table = pd.Series([sentance, 
                       original_target[5:-5].strip(), 
                       predicted_target[:-5].strip(), 
                       "%.3f" % sentence_bleu([hypothetic], reference, weights = [1])], 
                          index=['Sentance','Original','Predicted', 'BLEU'])
    
    print(table.to_string())
    print("==" * 50)

Test #1
Sentance              нет я работал у меня отпуск будет зимой
Original     нæ кусгæ кæнын мæнæн ме нцойуагъд зымæджы уыдзæн
Predicted                        æз дæр дæр дæр дæр нæ федтай
BLEU                                                    0.125
Test #2
Sentance     расскажите нам всё что знаете
Original     радзурут нын алцыдæр цы зонут
Predicted             нæ цы æрцыди дыл нæу
BLEU                                 0.200
Test #3
Sentance     у нас возле дома есть автобусная остановка
Original            нæ хæдзары цур ис автобусы æрлæууæн
Predicted                мæ фыд дыууæ нана уайтагъд ног
BLEU                                              0.000
Test #4
Sentance       он его нашёл
Original     уый йæ ссардта
Predicted         уый бон у
BLEU                  0.333
Test #5
Sentance        мои уши красные
Original     мæ хъустæ сырx сты
Predicted      дæ ном куыд алан
BLEU                      0.000
Test #6
Sentance      том ест корм для кошек
Original     том хæры гæдиты хæ

# Сохранение

In [25]:
def save_tokenizers(path, tokenizer_input, tokenizer_output):
    with open(f'{path[5:-4]}-tokenizer_input.pkl','wb') as f:
        pkl.dump(tokenizer_input, f)

    with open(f'{path[5:-4]}-tokenizer_output.pkl','wb') as f:
        pkl.dump(tokenizer_output, f)

    pkl.dump(tokenizer_input, open(f'{path[5:-4]}-tokenizer_input.pkl', 'wb'))
    pkl.dump(tokenizer_output, open(f'{path[5:-4]}-tokenizer_output.pkl', 'wb'))

In [26]:
def save_model_and_weights(model):
    model_json = model.to_json()
    
    with open(f'GRU-[{path[5:-4]}]-[Epochs={epochs}]-[LossFunction={loss_function}].json', "w") as json_file:
        json_file.write(model_json)
    
    model.save_weights(f'GRU-[{path[5:-4]}]-[Epochs={epochs}]-[LossFunction={loss_function}]-[weight].h5')

In [27]:
save_tokenizers(path, tokenizer_input, tokenizer_output)
save_model_and_weights(model)

# Загрузка и проверка

In [28]:
def load_model_and_weights():
    json_file = open(f'GRU-[{path[5:-4]}]-[Epochs={epochs}]-[LossFunction={loss_function}].json')
    loaded_model_json = json_file.read()
    json_file.close()
    model_loaded = model_from_json(loaded_model_json)

    model_loaded.load_weights(f'GRU-[{path[5:-4]}]-[Epochs={epochs}]-[LossFunction={loss_function}]-[weight].h5')
    
    return model_loaded

In [29]:
def load_tokenizers():
    with open(f'{path[5:-4]}-tokenizer_input.pkl','rb') as f:
        tokenizer_input = pkl.load(f)
    with open(f'{path[5:-4]}-tokenizer_output.pkl','rb') as f:
        tokenizer_output = pkl.load(f)

    return tokenizer_input, tokenizer_output

In [30]:
model_load = load_model_and_weights()
tokenizer_input_load, tokenizer_output_load = load_tokenizers()

In [31]:
encoder_model_load = Encoder.getModel(model_load)
decoder_model_load = Decoder.getModel(model_load)

In [32]:
reverse_word_map_input  = dict(map(reversed, tokenizer_input_load.word_index.items()))
reverse_word_map_target = dict(map(reversed, tokenizer_output_load.word_index.items()))

In [33]:
for i in range(4):
    sentance = X_test[i]
    original_target = y_test[i]
    input_seq = tokenizer_input.texts_to_sequences([sentance])
    pad_sequence = pad_sequences(input_seq, maxlen=30, padding='post')
    predicted_target = predict(pad_sequence, encoder_model_load, decoder_model_load, tokenizer_output_load, reverse_word_map_target)
    reference = original_target[5:-5].strip().split()
    hypothetic = predicted_target[:-5].strip().split()
    
    print(f'Test #{i + 1}')
    table = pd.Series([sentance, 
                       original_target[5:-5].strip(), 
                       predicted_target[:-5].strip(), 
                       "%.3f" % sentence_bleu([hypothetic], reference, weights = [1])], 
                          index=['Sentance','Original','Predicted', 'BLEU'])
    
    print(table.to_string())
    print("==" * 50)

Test #1
Sentance              нет я работал у меня отпуск будет зимой
Original     нæ кусгæ кæнын мæнæн ме нцойуагъд зымæджы уыдзæн
Predicted                        æз дæр дæр дæр дæр нæ федтай
BLEU                                                    0.125
Test #2
Sentance     расскажите нам всё что знаете
Original     радзурут нын алцыдæр цы зонут
Predicted             нæ цы æрцыди дыл нæу
BLEU                                 0.200
Test #3
Sentance     у нас возле дома есть автобусная остановка
Original            нæ хæдзары цур ис автобусы æрлæууæн
Predicted                мæ фыд дыууæ нана уайтагъд ног
BLEU                                              0.000
Test #4
Sentance       он его нашёл
Original     уый йæ ссардта
Predicted         уый бон у
BLEU                  0.333
